Saving memory size can greatly reduce the burden to your system memory while improve your analysis experience.
Here there are 3 steps to do it. 

In [ ]:
import os
import pandas as pd

#Garbage Collector
import gc

In [ ]:
#Read in the data
train_id = pd.read_csv('../input/train_identity.csv')
train_tran = pd.read_csv('../input/train_transaction.csv')

## 1. Downcasting Numeric Columns

In [ ]:
#This is a function that downcast the integer columns
def downcast_df_int_columns(df):
    list_of_columns = list(df.select_dtypes(include=["int32", "int64"]).columns)
        
    if len(list_of_columns)>=1:
        max_string_length = max([len(col) for col in list_of_columns]) # finds max string length for better status printing
        print("downcasting integers for:", list_of_columns, "\n")
        
        for col in list_of_columns:
            print("reduced memory usage for:  ", col.ljust(max_string_length+2)[:max_string_length+2],
                  "from", str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8), "to", end=" ")
            df[col] = pd.to_numeric(df[col], downcast="integer")
            print(str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8))
    else:
        print("no columns to downcast")
    
    gc.collect()
    
    print("done")

In [ ]:
#This is a function that downcast the float columns, if you have too many columns to adjust and do not want to see to many messages proceesing, you could comment our the print() columns
def downcast_df_float_columns(df):
    list_of_columns = list(df.select_dtypes(include=["float64"]).columns)
        
    if len(list_of_columns)>=1:
        max_string_length = max([len(col) for col in list_of_columns]) # finds max string length for better status printing
        print("downcasting float for:", list_of_columns, "\n")
        
        for col in list_of_columns:
            print("reduced memory usage for:  ", col.ljust(max_string_length+2)[:max_string_length+2],
                  "from", str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8), "to", end=" ")
            df[col] = pd.to_numeric(df[col], downcast="float")
            print(str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8))
    else:
        print("no columns to downcast")
    
    gc.collect()
    
    print("done")

In [ ]:
train_id.info(memory_usage="deep")

In [ ]:
train_tran.info(memory_usage="deep")

In [ ]:
#Reducing Size for the numeric columns for the train id part
downcast_df_int_columns(train_id)
downcast_df_float_columns(train_id)

In [ ]:
#Reducing Size for the numeric columns for the train transaction part
downcast_df_int_columns(train_tran)
downcast_df_float_columns(train_tran)

In [ ]:
train_id.info(memory_usage="deep")

In [ ]:
train_tran.info(memory_usage="deep")

As you can see, the tran_id dataframe has been downcasted from 161.9MB to 148.7MB (8.1%), while the trin_tran dataframe has been downcasted from 2.1GB to 1.2GB (42.9%). Of course, the more int and float column you have, the chance that these functions can better improve your performaces will be.

The idea behind the scene is pandas automatically read in your dataframe using int 64 or float 64, most of the time you do not need it to be this big, here is a size chart from StackOverflow:
**   Type      Capacity

   Int16 -- (-32,768 to +32,767)

   Int32 -- (-2,147,483,648 to +2,147,483,647)

   Int64 -- (-9,223,372,036,854,775,808 to +9,223,372,036,854,775,807)**
   
So most of the time, int16 or even int8 could do the job and thus save you the space. 

## 2.Downcasting the object column by converting them to categorical

In [ ]:
pd.set_option('display.max_columns', None)

train_id.head(10)

In [ ]:
#This is a function that convert a list of columns from object to categorical
def convert_columns_to_catg(df, column_list):
    for col in column_list:
        print("converting", col.ljust(30), "size: ", round(df[col].memory_usage(deep=True)*1e-6,2), end="\t")
        df[col] = df[col].astype("category")
        print("->\t", round(df[col].memory_usage(deep=True)*1e-6,2))

In [ ]:
#Picking some columns that seems to be able to convert to categorical
convert_columns_to_catg(train_id, ['id_12', 'id_16', 'id_30', 'id_31', 'id_33', 'id_34', 'id_34', 
                                  'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo'])

In [ ]:
train_id.info()

The idea behind the this is based on the fact that most of the column only takes few values and pandas can limit those values only to the few categorical value to save the memory. 

"Often in real-time, data includes the text columns, which are repetitive. Features like gender, country, and codes are always repetitive. These are the examples for categorical data.

Categorical variables can take on only a limited, and usually fixed number of possible values. Besides the fixed length, categorical data might have an order but cannot perform numerical operation. Categorical are a Pandas data type.

The categorical data type is useful in the following cases −

A string variable consisting of only a few different values. Converting such a string variable to a categorical variable will save some memory.

The lexical order of a variable is not the same as the logical order (“one”, “two”, “three”). By converting to a categorical and specifying an order on the categories, sorting and min/max will use the logical order instead of the lexical order.

As a signal to other python libraries that this column should be treated as a categorical variable (e.g. to use suitable statistical methods or plot types)."
(https://www.tutorialspoint.com/python_pandas/python_pandas_categorical_data)

## 3.Saving your dataframe as pickle file for fast read

In [ ]:
train_id.to_pickle("train_id.pkl")

print("train_identity.csv:", os.stat('../input/train_identity.csv').st_size * 1e-6)
print("train_id.pkl:", os.stat('train_id.pkl').st_size * 1e-6)

**Do not Forget to save your work. This way you can access the downcasted df everytime you load it!**

This code refers to Kaggle kernal: https://www.kaggle.com/frankherfert/tips-tricks-for-working-with-large-datasets?source=post_page---------------------------